# 목차
    1. [라이브러리 불러오기](#라이브러리-불러오기)
    2. [파일 불러오기 & API 키 받아오기](#파일--api-키-받아오기)
    3. [gpt 필드 전처리](#gpt)
    4. [gpt 필드 정리 & 저장](#생성-필드-합치고-저장하기)

# 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pydantic import BaseModel
from typing import List, Optional

# 파일 & API 키 받아오기

In [2]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'preprocessing')
data_file_path = os.path.join(data_directory, '전처리_호텔_results_20241010_113155.pickle')
df = pd.read_pickle(data_file_path)

In [3]:
# 환경변수에서 'OPENAPI_KEY' 값을 받아옴
# openapi_key = os.getenv('OPEN_API_KEY')

# GPT

In [4]:
df['description_1'] = '제목 : ' + df['title'].astype(str) + ', 상품 게시 시간 : ' + df['post_time'].astype(str) + ', 상세 설명 : ' + df['description'].astype(str)

In [31]:
# Pydantic을 사용해 응답 데이터를 구조화할 모델을 정의합니다.
class Fields(BaseModel):
    expiration_stdate: Optional[str]
    expiration_endate: Optional[str]
    room_type: Optional[str]
    head_count: Optional[int]
    shipping_fee: Optional[int]
    transaction_method: Optional[str]
    market_price: Optional[int]
    options: Optional[str]
    parking: Optional[bool]
    check_in_time: Optional[str]
    check_out_time: Optional[str]
    stay_type: Optional[str]
    foreign_yn: Optional[str]
    event: Optional[str]
    advertise : Optional[bool]

# 함수를 정의하여 데이터를 요청하고 응답을 처리합니다.
def extract_fields(description):
    prompt = {
        "role": "system", 
        "content": """
        너는 데이터 필드를 추출해주는 도우미야. 데이터를 받아서 필요한 필드를 15개로 제한해 JSON 형식으로 응답할거야. 
        설명에서 필요한 15개의 필드를 정확히 추출하고 나머지 필드들은 무시해줘. 필드는 오직 15개만 있어야 하고, 추가적인 세부 필드를 포함해서는 안 돼.
        NaN이나 정보가 없으면 해당 필드를 null로 설정해줘.
        필드는 아래 15개:
        1. expiration_stdate (해당 상품의 사용기한 중 시작 날짜 YYYY-MM-DD 형식)
        2. expiration_endate (해당 상품의 사용기한 중 마감 날짜 YYYY-MM-DD 형식)
        3. room_type (객실 유형)
        4. head_count (사용 인원수, int로만 알려줘)
        5. shipping_fee (배송비, 없으면 0)
        6. transaction_method (직거래/택배거래 여부, 카테고리는 4개 밖에 없어. '직거래', '택배거래', '둘 다 가능', '기타' 중 하나로 고정해서 분류해줘. '직거래'나 '택배거래'라는 키워드가 없으면 모두 '기타'로 분류해줘.)
        7. market_price (시중 가격, int로만 알려줘)
        8. options (호텔 옵션들 항목들을 콤마로 구분)
        9. parking (주차 가능 여부, True/False)
        10. check_in_time (체크인 시간,형식 : HH:MM, 입력된 값이 HH:MM 형식이 아니면 null로 설정)
        11. check_out_time (체크아웃 시간, 형식 : HH:MM, 입력된 값이 HH:MM 형식이 아니면 null로 설정)
        12. stay_type (숙소 형태, 카테고리는 호텔, 모텔, 리조트, 풀빌라, 펜션, 카테고리 이외는 모두 기타로 분류)
        13. foreign_yn (국내 숙소인지 외국 숙소인지 알려줘, 국내/외국)
        14. event (축제/콘서트 등 관련 이벤트가 포함되어있으면 축제,콘서트 이름을 써줘)
        15. advertise (광고글 여부,숙박 시설을 판매 및 양도하는 경우는 광고글이 아니야. 대리 구매나 숙박이 아닌 다른 걸 판매하는게 광고글이야, True/False)"""
    }

    query = {
        "role": "user",
        "content": f"""
        설명에서 필요한 15개의 필드를 추출하고 JSON 형식으로 반환해줘. 단, 필드는 15개 고정이야.
        
        설명: {description}
        """
    }

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 또는 "gpt-4" 사용 가능
            messages=[prompt, query],
            temperature=0,
            max_tokens=1000
        )

        # 응답을 받아 JSON으로 변환
        result = response['choices'][0]['message']['content']
        result = result.strip()  # 공백 제거

        # json으로 시작되는 쓸데없는 부분을 제거
        if result.startswith("```json"):
            result = result.replace("```json", "").strip()
        if result.endswith("```"):
            result = result.replace("```", "").strip()

        # Pydantic을 사용해 응답을 구조화된 데이터로 변환
        parsed_data = Fields.parse_raw(result)
        return parsed_data.dict()  # 딕셔너리로 반환

    except Exception as e:
        print("응답 파싱 중 오류 발생:", e)
        return {field: np.nan for field in Fields.__fields__}  # NaN 값으로 채운 딕셔너리 반환

In [32]:
# 멀티쓰레딩
def process_descriptions(descriptions):
    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(extract_fields, desc) for desc in descriptions]
        
        # tqdm으로 진행률 표시
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing descriptions"):
            results.append(future.result())
    
    return results

descriptions = df['description_1'].tolist()
processed_data = process_descriptions(descriptions)

Processing descriptions:  10%|▉         | 185/1913 [01:52<13:29,  2.13it/s]

응답 파싱 중 오류 발생: 1 validation error for Fields
market_price
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='null', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing


Processing descriptions:  62%|██████▏   | 1195/1913 [11:46<05:33,  2.15it/s]

응답 파싱 중 오류 발생: 1 validation error for Fields
market_price
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='null', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing


Processing descriptions:  90%|█████████ | 1726/1913 [16:52<02:05,  1.49it/s]

응답 파싱 중 오류 발생: 1 validation error for Fields
market_price
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='null', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing


Processing descriptions: 100%|██████████| 1913/1913 [18:38<00:00,  1.71it/s]


# 생성 필드 합치고 저장하기

In [33]:
processed_data

[{'expiration_stdate': None,
  'expiration_endate': None,
  'room_type': '스탠다드더블룸',
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '기타',
  'market_price': None,
  'options': None,
  'parking': False,
  'check_in_time': None,
  'check_out_time': None,
  'stay_type': '호텔',
  'foreign_yn': '국내',
  'event': None,
  'advertise': False},
 {'expiration_stdate': None,
  'expiration_endate': None,
  'room_type': None,
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '기타',
  'market_price': None,
  'options': None,
  'parking': False,
  'check_in_time': None,
  'check_out_time': None,
  'stay_type': '기타',
  'foreign_yn': '국내',
  'event': None,
  'advertise': False},
 {'expiration_stdate': None,
  'expiration_endate': None,
  'room_type': '객실만',
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '기타',
  'market_price': None,
  'options': None,
  'parking': False,
  'check_in_time': '15:00',
  'check_out_time': '12:00',
  'stay_type': '호텔',
  '

In [40]:
df1 = pd.DataFrame(processed_data)
df1

,expiration_stdate,expiration_endate,room_type,head_count,shipping_fee,transaction_method,market_price,options,parking,check_in_time,check_out_time,stay_type,foreign_yn,event,advertise
0,None,None,스탠다드더블룸,NaN,0.0,기타,NaN,None,False,None,None,호텔,국내,None,False
1,None,None,None,NaN,0.0,기타,NaN,None,False,None,None,기타,국내,None,False
2,None,None,객실만,NaN,0.0,기타,NaN,None,False,15:00,12:00,호텔,국내,None,False
3,2024-10-10,2024-10-12,None,NaN,0.0,기타,NaN,None,False,None,None,호텔,국내,OKTOBERFEST,False
4,null,2049-02-28,None,NaN,0.0,직거래,NaN,None,False,None,None,호텔,국내,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,None,None,Gorgeous Ocean Twin,3.0,0.0,기타,NaN,None,False,None,None,호텔,국내,None,False
1909,None,None,더블베드+싱글베드,3.0,0.0,기타,550000.0,None,False,None,None,호텔,국내,None,False
1910,None,None,None,NaN,0.0,기타,NaN,None,False,None,None,호텔,국내,None,True
1911,None,None,None,NaN,0.0,기타,NaN,None,False,None,None,호텔,국내,None,False


In [35]:
df1['check_in_time'].unique()

array([None, '15:00', '20:00', '16:00', '18:00', '13:00', '10:00', nan,
       '17:00', 'null', '00:00', '09:30', '21:00', '14:00', '23:00',
       '12:00', '11:00', '22:00', 'HH:MM 형식이 아니어서 null', '03:00', '05:00',
       '15:30', '19:00', '04:00', '오전', '09:00'], dtype=object)

In [36]:
df1['check_out_time'].unique()

array([None, '12:00', '16:00', '14:00', '13:00', 'null', '10:00', '11:00',
       '01:00', nan, '00:00', '2:00', '18:00', '10:30', '레이트체크아웃',
       '15:00', '23:00', '21:00', '22:00', '넉넉', '17:00', '11:30'],
      dtype=object)

In [37]:
df1['transaction_method'].unique()

array(['기타', '직거래', '택배거래', '양도/판매', '둘 다 가능', nan, '직거래 또는 택배', '외국'],
      dtype=object)

In [41]:
# df와 df1을 열 기준으로 merge
merged_df = pd.merge(df, df1, left_index=True, right_index=True)
# 필요 없는 column 삭제
merged_df = merged_df.drop(['expiration_date', 'market_price_x', 'capacity',
       'parking_x', 'options_x', 'check_in_out_time', 'shipping_fee_x',
       'transaction_location', 'transaction_method_x', 'city','city_dong'], axis=1)
# column이름 재정의
merged_df.columns = merged_df.columns.str.replace('_y', '')
merged_df

,platform,original_link,post_time,title,view_count,like_count,price,images,description,category,...,transaction_method,market_price,options,parking,check_in_time,check_out_time,stay_type,foreignn,event,advertise
0,당근마켓,https://www.daangn.com/articles/845853060,2024-10-10 10:27:51.013,10/11-12 금토) 호텔케니여수 스탠다드더블룸 양도해요,7,0,44000,https://img.kr.gcp-karroter.net/origin/article...,못 가게되서요 양도합니다,티켓/교환권,...,기타,NaN,None,False,None,None,호텔,국내,None,False
1,당근마켓,https://www.daangn.com/articles/845851669,2024-10-10 10:24:51.013,●농심호텔 허심청브로이 옥토버페스트 수제맥주축제 OKTOBERFEST 티켓,29,2,28000,https://img.kr.gcp-karroter.net/origin/article...,농심호텔\n허심청브로이 옥토버페스트 수제맥주축제 \nOKTOBERFEST\n2024...,티켓/교환권,...,기타,NaN,None,False,None,None,기타,국내,None,False
2,당근마켓,https://www.daangn.com/articles/845850450,2024-10-10 10:22:51.013,호텔스카이파크센트럴 판교 10/12(토)~13(일),21,0,100000,https://img.kr.gcp-karroter.net/origin/article...,환불불가 상품으로\n개인사정으로 못가게 되어서 팝니다\n객실만이에요\n조식 미포함\...,티켓/교환권,...,기타,NaN,None,False,15:00,12:00,호텔,국내,None,False
3,당근마켓,https://www.daangn.com/articles/845834566,2024-10-10 09:54:51.013,신라호텔 더 파크뷰 레스토랑 식사권 2매 판매합니다,87,6,360000,https://img.kr.gcp-karroter.net/origin/article...,신라호텔 더 파크뷰 레스토랑 식사권 2매 판매합니다,티켓/교환권,...,기타,NaN,None,False,None,None,호텔,국내,OKTOBERFEST,False
4,당근마켓,https://www.daangn.com/articles/845832050,2024-10-10 09:54:51.013,"호텔 드포레 ""주말"" 숙박권 (~11-30) [네이처파크 무료]",354,33,100000,https://img.kr.gcp-karroter.net/origin/article...,스파밸리 호텔\n호텔 드포레\n친환경 힐링 숲속호텔\n(네이처파크 무료이용)\n주중...,티켓/교환권,...,직거래,NaN,None,False,None,None,호텔,국내,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,중고나라,https://web.joongna.com/product/177054727,2024-10-10 11:31:55.993,8월 15일 ~17일 (2박)세인트 존스 경포 호텔 고져스 오션 트윈,31,0,800000,/cafe-article-data/live/2024/07/18/1067047568/...,세인트존스경포호텔 고져스 오션 트윈 객실 숙박 -예약일정 : 08/15 토 ~ 08...,여행/숙박이용권,...,기타,NaN,None,False,None,None,호텔,국내,None,False
1909,중고나라,https://web.joongna.com/product/177054641,2024-10-10 11:31:55.993,센츄리온 호텔 우에노 7.19-21 (2박) 급매!,6,0,150000,/cafe-article-data/live/2024/07/18/1067047466/...,https://search.naver.com/search.naver?where=ne...,여행/숙박이용권,...,기타,550000.0,None,False,None,None,호텔,국내,None,False
1910,중고나라,https://web.joongna.com/product/177046630,2024-10-10 11:31:55.993,마카오 호텔 숙박권 양도합니다,30,0,90000,https://img2.joongna.com/media/original/2024/0...,세나도광장 걸어서 3-5분 거리에있는 위치 아주 좋은 숙소입니다 ! 16일에 체크인...,여행/숙박이용권,...,기타,NaN,None,False,None,None,호텔,국내,None,True
1911,중고나라,https://web.joongna.com/product/177030077,2024-10-10 11:31:55.993,당일 롯데호텔 숙박권,63,0,200000,https://img2.joongna.com/media/original/2024/0...,오늘날짜 숙박권 입니다 저렴하게 판매해요,여행/숙박이용권,...,기타,NaN,None,False,None,None,호텔,국내,None,False


In [42]:
merged_df.to_csv('description_complete3.csv',encoding='utf-8-sig')